<a href="https://colab.research.google.com/github/mavho/CSE143/blob/master/A2_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Installs the correct packages.

First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20 and TensorFlow ≥2.0.

In [3]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    #pip install -q -U tensorflow-addons
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.test.is_gpu_available():
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

TensorFlow 2.x selected.
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


# Sentiment Analysis

Loads the data into a train and test set.
X_train is a list of reviews, represented as numpy arrays of integers

In [4]:
(X_train, y_test), (X_valid, y_test) = keras.datasets.imdb.load_data()

17465344/17464789 [==============================] - 1s 0us/step


Below is an example of how to do custom preprocessing

In [5]:
import tensorflow_datasets as tfds
train_data, dev_data, test_data = tfds.load("imdb_reviews", split=('train[:60%]', 'train[60%:]', 'test'), as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteRR62ZS/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteRR62ZS/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteRR62ZS/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

Construct the vocaulary, counts occurence of each word.

In [0]:
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in train_data.batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

Look at only the first 10000 most common words in the vocab

In [0]:
vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]

In [0]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

Creating the final training set.
We batch the reviews, and using the preprocess function, encode them.

In [0]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = train_data.batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)
dev_set = dev_data.batch(32).map(preprocess)
dev_set = dev_set.map(encode_words).prefetch(1)

What the train set looks like...

Creates the model and trains it.

In [12]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.SimpleRNN(128 ,activation ='relu',dropout = .15),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, validation_data=dev_set, epochs=10)

Epoch 1/10
469/469 [==============================] - 37s 79ms/step - loss: 0.6955 - accuracy: 0.5171 - val_loss: 0.6889 - val_accuracy: 0.5378
Epoch 2/10
469/469 [==============================] - 35s 76ms/step - loss: 0.6234 - accuracy: 0.6675 - val_loss: 0.6089 - val_accuracy: 0.6706
Epoch 3/10
469/469 [==============================] - 34s 73ms/step - loss: 0.4142 - accuracy: 0.8119 - val_loss: 0.6938 - val_accuracy: 0.6542
Epoch 4/10
469/469 [==============================] - 35s 75ms/step - loss: 0.2748 - accuracy: 0.8853 - val_loss: 0.8305 - val_accuracy: 0.6181
Epoch 5/10
469/469 [==============================] - 35s 75ms/step - loss: 0.1990 - accuracy: 0.9225 - val_loss: 1.0065 - val_accuracy: 0.6062
Epoch 6/10
469/469 [==============================] - 35s 74ms/step - loss: 0.1289 - accuracy: 0.9519 - val_loss: 1.1985 - val_accuracy: 0.6059
Epoch 7/10
469/469 [==============================] - 35s 75ms/step - loss: 0.1617 - accuracy: 0.9409 - val_loss: 1.2045 - val_accuracy:

In [13]:
history.model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         1408000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 128)               32896     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,441,025
Trainable params: 1,441,025
Non-trainable params: 0
_________________________________________________________________


In [0]:
test_set = test_data.batch(512).map(preprocess)
test_set = test_set.map(encode_words).prefetch(1)



embedds the test data, this is how well it performs?

In [21]:
loss,accuracy = model.evaluate(test_set,steps=10)

10/10 [==============================] - 1s 77ms/step - loss: 1.8291 - accuracy: 0.6770
